# Bathymetry Subtidal
Tests subtidal bathymetry functions of `bathymetry.py`.

In [31]:
from pathlib import Path
import sys
import time

import ee
import geemap
from geojson import Polygon

sys.path.append(str(Path.cwd().parent.parent))

from eobathymetry import bathymetry

In [37]:
Map = geemap.Map(center=(52.97, 4.74), zoom=11)
bounds = ee.Geometry(Polygon([[
    (4.574518432617185, 52.91662291147939),
    (4.90548156738281, 52.91662291147939),
    (4.90548156738281, 53.02331125248889),
    (4.574518432617185, 53.02331125248889),
    (4.574518432617185, 52.91662291147939)
]]))
# scale = Map.get_scale()
scale = 30
start_date = "2020-01-01"
stop_date = "2021-01-01"

sdb = bathymetry.Bathymetry()
Map.addLayer(bounds)
Map

Map(center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [36]:
# Run interactively without neighborhood search
image = sdb.compute_inverse_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale*5,
    missions=["L8", "S2"],
    filter_masked=True,
    skip_neighborhood_search=True
)
Map.addLayer(image.reproject(ee.Projection('EPSG:3857').atScale(90)))  # without projection, takes default projection
Map

Map(bottom=171160.0, center=[52.97, 4.74], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [41]:
# Interactive times out, let's use the ee.batch.Export class' static methods to create assets
image = sdb.compute_inverse_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale*5,
    missions=["L8", "S2"],
    filter_masked=True,
    skip_neighborhood_search=False # adds a lot of computation time
)
user_name = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
asset_id = f"users/{user_name}/tests/test_inverse_depth"
try:
    ee.data.deleteAsset(asset_id)
except ee.EEException as e:
    if f"Asset \"projects/earthengine-legacy/assets/{asset_id}\" not found." not in str(e):
        raise e
    else:
        print(f"Asset {asset_id} not found")
    
ee.data.create_assets(asset_ids=["/".join(asset_id.split("/")[:-1])], asset_type="Folder", mk_parents=True)
task = ee.batch.Export.image.toAsset(
    image,
    description="test_compute_inverse_depth",
    assetId=asset_id,
    scale=5*scale,
    maxPixels=1e11
)

Asset users/jaapel/tests/test_inverse_depth not found
Asset users/jaapel/tests already exists.


In [42]:
task.start()

In [43]:
task_status = task.status()
while task_status['state'] in ['READY', 'RUNNING']:
    print(task_status)
    time.sleep(30)
    task_status = task.status()
else:
    print(task.status())

{'state': 'READY', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632232934714, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632232955772, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632232955772, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'descrip

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632233555729, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632233705734, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632233705734, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632234305736, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632234455736, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632234455736, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632235205738, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632235205738, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632235205738, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632235955735, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632235955735, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632235955735, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

...

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632387209013, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632387209013, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632387209013, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632387959009, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632387959009, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632387959009, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632388709022, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632388709022, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632388709022, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632389459014, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632389459014, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632389459014, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632390209013, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632390209013, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{'state': 'RUNNING', 'description': 'test_compute_inverse_depth', 'creation_timestamp_ms': 1632232934714, 'update_timestamp_ms': 1632390209013, 'start_timestamp_ms': 1632232955628, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'GCPKSW62QV4VOIS533YWUESH', 'name': 'projects/earthengine-legacy/operations/GCPKSW62QV4VOIS533YWUESH'}
{

In [ ]:
image_subtidal = ee.Image(asset_id)
Map.addLayer(image_subtidal, {"gamma": 1.2}, 'RGB_raw', True)
Map